In [222]:

import mysql
import mysql.connector
import csv
import pandas as pd
import numpy as np
import scipy
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support

cnx = mysql.connector.connect(user='ctsimaan',password='SeniorProject490',
                              host='nostradomicile-data.c6x7vypetdqh.us-west-2.rds.amazonaws.com',
                              database='PyZillow_Data')
cursor = cnx.cursor()

In [223]:
df_pd = pd.read_sql('SELECT*FROM PyZillow_Data.home_data Where zip = 27705', con=cnx)
frames = [df_pd,df_pd]
df_pd = pd.concat(frames)

print ('loaded dataframe from MySQL. records:', len(df_pd))
print(df_pd)

('loaded dataframe from MySQL. records:', 712)
              street_address    zip    city state      home_type  bedrooms  \
0                1 Dinara Dr  27705  Durham    NC    Condominium         3   
1          10 Blue Bottle Ln  27705  Durham    NC  Miscellaneous         3   
2            10 Marchmont Ct  27705  Durham    NC   SingleFamily         3   
3              10 Windrow Pl  27705  Durham    NC   SingleFamily         4   
4           100 Solterra Way  27705  Durham    NC   SingleFamily         3   
5          100 Sturdivant Rd  27705  Durham    NC   SingleFamily         4   
6          1001 Rosehill Ave  27705  Durham    NC   SingleFamily         3   
7           1009 Oakland Ave  27705  Durham    NC   SingleFamily         4   
8          1009 Rosehill Ave  27705  Durham    NC   SingleFamily         3   
9         101 Brook Chase Ln  27705  Durham    NC   SingleFamily         5   
10          101 Hay Sedge Ct  27705  Durham    NC   SingleFamily         4   
11             10

In [224]:
new = df_pd.filter(['sold_binary', 'home_type', 'bedrooms', 'bathrooms', 'finished_sq_footage',
       'lot_size_sq_footage','year_built','last_sale_price','neighborhood', 'school_district'], axis = 1)
#d = {'sold_binary':0, 'bedrooms':5,'bathrooms':3,'finished_sq_footage':2000,
     #'lot_size_sq_footage':7405.0,'year_built':1990, 'last_sale_price':250000.0,'home_type':"SingleFamily",'neighborhood':
     #None, 'school_district':None}
#d = pd.DataFrame(data = d, index=[356])
#addData = [new,d]
#new = pd.concat(addData)
df = pd.DataFrame(data = new)
cols_to_transform = [ 'home_type','neighborhood','school_district']
df_dum = pd.get_dummies(data=df,columns = cols_to_transform)
df_dum = df_dum.fillna(df_dum.mean())

In [225]:
array = df_dum.values
print(df_dum)

     sold_binary  bedrooms  bathrooms  finished_sq_footage  \
0              0         3   3.000000                 1830   
1              0         3   3.000000                 1754   
2              0         3   3.000000                 3132   
3              0         4   3.000000                 3745   
4              0         3   2.000000                 1975   
5              0         4   4.000000                 3127   
6              0         3   3.000000                 2158   
7              0         4   2.000000                 1938   
8              0         3   1.000000                 1350   
9              0         5   3.000000                 3456   
10             1         4   3.000000                 2500   
11             1         3   3.000000                 2126   
12             1         4   3.000000                 3389   
13             1         3   3.000000                 1889   
14             0         3   2.000000                 1761   
15      

In [226]:
X = array[:,1:35]
Y = array[:,0]


In [227]:
train, test = train_test_split(df_dum, test_size = 0.3)

In [228]:
features = df_dum.columns[1:35]
print(features)

Index([                                                  u'bedrooms',
                                                        u'bathrooms',
                                              u'finished_sq_footage',
                                              u'lot_size_sq_footage',
                                                       u'year_built',
                                                  u'last_sale_price',
                                            u'home_type_Condominium',
                                                 u'home_type_Duplex',
                                          u'home_type_Miscellaneous',
                                        u'home_type_MultiFamily2To4',
                                           u'home_type_SingleFamily',
                                              u'home_type_Townhouse',
                                  u'neighborhood_124 - North Durham',
                          u'neighborhood_Across From Durham Academy',
                    

In [229]:
seed = 7
max_features = 3
#rf = RandomForestClassifier(n_estimators=100, max_features="auto",oob_score = True, 
                           # n_jobs = -1,random_state =50,min_samples_leaf = 50)
rf = RandomForestClassifier(n_estimators=100, max_features=max_features)
y, _ = pd.factorize(train['sold_binary'])
#y = train['sold_binary']
rf.fit(train[features], y)
#rf.fit(df_dum,y)
preds = rf.predict(test[features])
#preds = rf.predict(df_dum)
kfold = model_selection.KFold(n_splits=10, random_state=seed)
results = model_selection.cross_val_score(rf, test[features], test['sold_binary'], cv=kfold)
print(precision_recall_fscore_support(test['sold_binary'], preds, average='micro'))
print(results.mean())
print "Features sorted by their score:"
print sorted(zip(map(lambda x: round(x, 4), rf.feature_importances_), features), 
             reverse=True)


# initialize


(0.89719626168224298, 0.89719626168224298, 0.89719626168224298, None)
0.630735930736
Features sorted by their score:
[(0.2252, 'finished_sq_footage'), (0.2122, 'lot_size_sq_footage'), (0.2019, 'year_built'), (0.2002, 'last_sale_price'), (0.0454, 'bedrooms'), (0.0453, 'bathrooms'), (0.0133, u'neighborhood_None'), (0.0125, u'school_district_None'), (0.0055, u'home_type_SingleFamily'), (0.0048, u'home_type_Townhouse'), (0.0038, u'neighborhood_American Village'), (0.0034, u'school_district_Durham'), (0.003, u'school_district_DURHAM PUBLIC SCHOOLS'), (0.003, u'neighborhood_The Carillion Woods'), (0.0026, u'neighborhood_Garrett Farms'), (0.0021, u'home_type_Condominium'), (0.0017, u'school_district_Durham Public Schools (Jordan High School)'), (0.0017, u'school_district_Durham County'), (0.0015, u'neighborhood_124 - North Durham'), (0.0014, u'neighborhood_Across From Durham Academy'), (0.0012, u'neighborhood_Watts-Hillandale'), (0.0012, u'neighborhood_Watts Hospital Hillandale'), (0.0012, u'

In [230]:
print(preds)

[1 1 0 0 0 0 1 0 1 0 0 0 0 0 1 1 0 0 1 0 0 1 0 0 0 0 0 0 1 1 1 1 1 0 1 1 0
 0 1 0 0 0 1 0 0 0 1 1 0 0 1 0 1 0 0 0 1 0 0 1 1 1 0 0 0 0 0 1 1 1 1 0 1 0
 0 0 0 0 0 0 0 1 1 1 0 0 0 1 0 1 0 0 0 1 0 0 0 1 1 0 0 0 1 1 1 1 0 0 0 1 1
 0 1 0 1 1 1 0 0 0 1 0 0 1 0 1 0 0 1 0 0 1 0 1 1 0 1 0 0 0 0 0 0 1 1 0 0 0
 0 0 0 0 0 1 1 0 0 0 0 0 1 0 1 0 1 1 0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 1 1 0 1
 0 0 1 0 1 0 1 1 0 0 0 1 1 0 0 1 1 1 0 1 0 1 1 1 0 1 1 0 1]
